In [1]:
import numpy as np
from transformers import CLIPTextModel, CLIPTokenizer
import pandas as pd

# Concept Vector of Violence

In [2]:
dir_ = "CompVis/stable-diffusion-v1-4"
tokenizer = CLIPTokenizer.from_pretrained(dir_, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(dir_, subfolder="text_encoder").to('cuda')
num_samples = 5
df = pd.read_csv('./data/Prompts_For_ConceptVector/Violence_30.csv')
violence_text=[]
for _, row in df.iterrows():
    prompt = [f"{row.prompt}"]*num_samples
    text_input = tokenizer(prompt ,padding="max_length", max_length=77,  truncation=True,  return_tensors="pt")
    embed = text_encoder(text_input.input_ids.to('cuda'), return_dict=True)[0] 
    violence_text.extend(embed.detach().cpu().numpy())
violence_text = np.array(violence_text)
NoViolence_text = []
df = pd.read_csv('./data/Prompts_For_ConceptVector/Violence_30.csv')
for _, row in df.iterrows():
    prompt = [f"{row.prompt1}"]*num_samples
    text_input = tokenizer(prompt ,padding="max_length", max_length=77,  truncation=True,  return_tensors="pt")
    embed = text_encoder(text_input.input_ids.to('cuda'), return_dict=True)[0] 
    NoViolence_text.extend(embed.detach().cpu().numpy())
NoViolence_text = np.array(NoViolence_text)

In [3]:
vec = np.mean(violence_text - NoViolence_text, axis=0)

# Concept Vector of Nudity

In [68]:
dir_ = "CompVis/stable-diffusion-v1-4"
tokenizer = CLIPTokenizer.from_pretrained(dir_, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(dir_, subfolder="text_encoder").to('cuda')
num_samples = 5
df = pd.read_csv('./data/Prompts_For_ConceptVector/nudity_prompt.csv')
nudity_text=[]
for _, row in df.iterrows():
    if row.clothes != 'None':
        if row.people == 'None':
            prompt = [f"{row.nudity}, {row.location}"]*num_samples
        else:
            prompt = [f"{row.nudity} {row.people}, {row.location}"]*num_samples
    else:
        prompt = [f"{row.nudity} {row.people}, {row.location}"]*num_samples
    text_input = tokenizer(prompt ,padding="max_length", max_length=77,  truncation=True,  return_tensors="pt")
    embed = text_encoder(text_input.input_ids.to('cuda'), return_dict=True)[0] 
    nudity_text.extend(embed.detach().cpu().numpy())
nudity_text = np.array(nudity_text)

NoNudity_text = []
df = pd.read_csv('../data/Prompts_For_ConceptVector/NoNudity_prompt.csv')
for _, row in df.iterrows():
    prompt = [f"{row.prompt}"]*num_samples
    text_input = tokenizer(prompt ,padding="max_length", max_length=77,  truncation=True,  return_tensors="pt")
    embed = text_encoder(text_input.input_ids.to('cuda'), return_dict=True)[0] 
    NoNudity_text.extend(embed.detach().cpu().numpy())
NoNudity_text = np.array(NoNudity_text)

### Why did the indices of fail occur?
<br>

To ensure the quality of nudity prompts, we utilize SD to generate 5 images for each prompt. Subsequently, we employ the NudeNet Detector to identify images without nudity. To obtain the concept vector, we discard the failed indices.

In [63]:
fail = [134, 15, 16, 147, 151, 35, 38, 174, 55, 62, 202, 204, 91, 93, 95, 96, 98, 106, 109]
nudity_text = np.delete(nudity_text, fail, axis=0)
NoNudity_text = np.delete(NoNudity_text, fail, axis=0)
vec = np.mean(nudity_text - NoNudity_text, axis=0)